04/02/24 JD

Goal: There are 20 test1 control probes. Find their location, and find the matching sequential rounds that share the same bit.

Note: run this on `encoder`, using kernel `probe_design`

In [1]:
import numpy as np
import pandas as pd
import os
import sys,os
sys.path.append(r'/mnt/tscc/wed009/WholeGenome/scripts/LibraryDesign3/')
import LibraryDesigner as ld
import LibraryTools as lt

# Get codebook

In [2]:
## read in codebook
df_pilot = pd.read_csv(r'/mnt/tscc/wed009/WholeGenome/data/FromBlair/pilot_encoding1.csv')
df_pilot

,Unnamed: 0.1,Unnamed: 0,block,bin,subblock,batch,locus,Bit_000,Bit_001,Bit_002,...,Bit_1270,Bit_1271,Bit_1272,Bit_1273,Bit_1274,Bit_1275,Bit_1276,Bit_1277,Bit_1278,Bit_1279
0,0,0,0,0,1,0,chr1_00000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,0,1,1,0,chr1_00001,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,2,0,2,1,0,chr1_00002,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,3,0,3,1,0,chr1_00003,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,4,0,4,1,0,chr1_00004,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7795,123895,123895,2064,55,4,0,chrY_01714,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7796,123896,123896,2064,56,4,0,chrY_01715,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7797,123897,123897,2064,57,4,0,chrY_01716,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7798,123898,123898,2064,58,4,0,chrY_01717,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
df_adt = pd.read_csv(r'/mnt/tscc/wed009/WholeGenome/data/FromBlair/Adapters.csv',header=None)
names_adt = list(df_adt[0].str.split('_i').str.get(0))
seqs_adt = [lt.seqrc(sq[:20])for sq in df_adt[1]]

In [4]:
df_adt

,0,1,2,3,4,5,6,7,8,9,10
0,NDB_784_i1_-2xStv_82rc,tagaaatatccgagtggcatTTTGCACTGCCGTCCTTGACTTTGCA...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A1,P1
1,NDB_755_i2_-2xStv_1rc,tgcgacatgttatgctaggcGATCCGATTGGAACCGTCCCGATCCG...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A2,P1
2,NDB_759_i3_-2xStv_79rc,tgactgtcgttgactctcggTGCGAACTGTCCGGCTTTCATGCGAA...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A3,P1
3,NDB_826_i4_-2xStv_82rc,atatgtcgatgtccttagacTTTGCACTGCCGTCCTTGACTTTGCA...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A4,P1
4,NDB_578_i5_-2xStv_1rc,aggcgatacatgagaggcgtGATCCGATTGGAACCGTCCCGATCCG...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A5,P1
...,...,...,...,...,...,...,...,...,...,...,...
292,NDB_368_i293_-2xStv_1rc,ctgacgctgaagtgcctagtGATCCGATTGGAACCGTCCCGATCCG...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
293,NDB_693_i294_-2xStv_79rc,attggcaagcgttgccctttTGCGAACTGTCCGGCTTTCATGCGAA...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
294,NDB_955_i295_-2xStv_82rc,gccgcacgaatgtgatagtaTTTGCACTGCCGTCCTTGACTTTGCA...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
295,NDB_707_i296_-2xStv_1rc,tccgctaagcgcggctgataGATCCGATTGGAACCGTCCCGATCCG...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## dictionary for adapters
index (key) : (NDB, sequence, is_combinatory) (values)

In [5]:
df_pilot_codebook = np.array(df_pilot.iloc[:, 7:].values,dtype=int)
nzRs = np.where(np.sum(df_pilot_codebook,axis=0)>0)[0]
is_combinatorial = np.zeros(1280)
is_combinatorial[:160]=1
is_combinatorial[160:(160+480)]=0
is_combinatorial[(160+480):]=1
names_loc = list(df_pilot['locus'])
dic_readout_seqs = dict(zip(nzRs,zip(names_adt,seqs_adt,is_combinatorial[nzRs])))

In [6]:
# dictionary for the adapters
dic_readout_seqs

{0: ('NDB_784', 'atgccactcggatatttcta', 1.0),
 1: ('NDB_755', 'gcctagcataacatgtcgca', 1.0),
 2: ('NDB_759', 'ccgagagtcaacgacagtca', 1.0),
 3: ('NDB_826', 'gtctaaggacatcgacatat', 1.0),
 4: ('NDB_578', 'acgcctctcatgtatcgcct', 1.0),
 5: ('NDB_795', 'caaacgtgtaacagacgccc', 1.0),
 6: ('NDB_376', 'atcaacgtacccacaaagtc', 1.0),
 7: ('NDB_713', 'ttcggaaagtgacaaccacc', 1.0),
 8: ('NDB_777', 'taacgaaagcaaattcgccg', 1.0),
 9: ('NDB_865', 'tcacatatccgaccgggatc', 1.0),
 10: ('NDB_533', 'tgtaccagtacgcagccgac', 1.0),
 11: ('NDB_801', 'caatgatcgtggacaccgac', 1.0),
 12: ('NDB_817', 'aatgcttagtcaacgagttc', 1.0),
 13: ('NDB_725', 'ctaaatcgatgccgtggtag', 1.0),
 14: ('NDB_807', 'gttcacgaagcagcataccg', 1.0),
 15: ('NDB_652', 'ggtacctactaaagcgccta', 1.0),
 16: ('NDB_710', 'taccgcggcttagtagttga', 1.0),
 17: ('NDB_690', 'accgatttatgaggccgttc', 1.0),
 18: ('NDB_718', 'cataaggatcctgcaatacg', 1.0),
 19: ('NDB_971', 'taaggccaccttgcgataag', 1.0),
 160: ('NDB_699', 'acacgctataaagggtcgat', 0.0),
 161: ('NDB_847', 'cca

In [7]:
len(dic_readout_seqs)

160

## dictionary for V1 regions
region (key): 5 bit (values)

In [8]:
dic_code_final = {}
for ielem in range(len(names_loc)):
    nm_loc = names_loc[ielem]
    cd_loc = np.where(df_pilot_codebook[ielem])[0]
    cd_locf = [ef for e in cd_loc 
               for ef in [e]*(2 if dic_readout_seqs[e][-1]==0 else 1)] ## repeat twice the seqential
    dic_code_final[nm_loc] = cd_locf

In [9]:
dic_code_final

{'chr1_00000': [160, 160, 644, 645, 649, 651],
 'chr1_00001': [161, 161, 644, 645, 649, 651],
 'chr1_00002': [162, 162, 644, 645, 649, 651],
 'chr1_00003': [163, 163, 644, 645, 649, 651],
 'chr1_00004': [164, 164, 644, 645, 649, 651],
 'chr1_00005': [165, 165, 644, 645, 649, 651],
 'chr1_00006': [166, 166, 644, 645, 649, 651],
 'chr1_00007': [167, 167, 644, 645, 649, 651],
 'chr1_00008': [168, 168, 644, 645, 649, 651],
 'chr1_00009': [169, 169, 644, 645, 649, 651],
 'chr1_00010': [170, 170, 644, 645, 649, 651],
 'chr1_00011': [171, 171, 644, 645, 649, 651],
 'chr1_00012': [172, 172, 804, 805, 809, 811],
 'chr1_00013': [173, 173, 804, 805, 809, 811],
 'chr1_00014': [174, 174, 804, 805, 809, 811],
 'chr1_00015': [175, 175, 804, 805, 809, 811],
 'chr1_00016': [176, 176, 804, 805, 809, 811],
 'chr1_00017': [177, 177, 804, 805, 809, 811],
 'chr1_00018': [178, 178, 804, 805, 809, 811],
 'chr1_00019': [179, 179, 804, 805, 809, 811],
 'chr1_00020': [180, 180, 804, 805, 809, 811],
 'chr1_00021'

# Get the 20 control regions
round 54, 56-64 are the control runs (total 10 rounds)

only Cy5 and Cy3 are `test1` controls

In [10]:
pilot_ctrls = pd.read_csv(r'/mnt/tscc/wed009/WholeGenome/data/design/PilotControls.csv')
pilot_ctrls

,round,valve,batch,adapter_mix,folder_name,identity
0,54,28,1,P2_F4_5_6,H54_P2_F4_5_6,test1
1,56,2,3,P2_F10_11_12,H56_P2_F10_11_12,test1
2,57,3,3,P2_G4_5_6,H57_P2_G4_5_6,test1
3,58,4,3,P2_G10_11_12,H58_P2_G10_11_12,test1
4,59,5,3,P2_H4_5_6,H59_P2_H4_5_6,test1
5,60,6,3,P3_C4_5_6,H60_P3_C4_5_6,test1
6,61,7,3,P3_A4_5_6,H61_P3_A4_5_6,test1
7,62,8,3,P3_A10_11_12,H62_P3_A10_11_12,test1
8,63,9,3,P3_B4_5_6,H63_P3_B4_5_6,test1
9,64,10,3,P3_B10_11_12,H64_P3_B10_11_12,test1


In [11]:
# build a dictionary where 
# keys: the plate_valve ID
# values: index in df_adapter
pilot_ctrls['folder_name'].values

array(['H54_P2_F4_5_6', 'H56_P2_F10_11_12', 'H57_P2_G4_5_6',
       'H58_P2_G10_11_12', 'H59_P2_H4_5_6', 'H60_P3_C4_5_6',
       'H61_P3_A4_5_6', 'H62_P3_A10_11_12', 'H63_P3_B4_5_6',
       'H64_P3_B10_11_12'], dtype=object)

In [12]:
ctrl_keys = ['_'.join(s.split('_')[1:3]).replace('4', '5').replace('10', '11') for s in pilot_ctrls['folder_name'].values] + \
['_'.join(s.split('_')[1:3]).replace('4', '6').replace('10', '12') for s in pilot_ctrls['folder_name'].values] 

In [13]:
ctrl_keys

['P2_F5',
 'P2_F11',
 'P2_G5',
 'P2_G11',
 'P2_H5',
 'P3_C5',
 'P3_A5',
 'P3_A11',
 'P3_B5',
 'P3_B11',
 'P2_F6',
 'P2_F12',
 'P2_G6',
 'P2_G12',
 'P2_H6',
 'P3_C6',
 'P3_A6',
 'P3_A12',
 'P3_B6',
 'P3_B12']

In [14]:
ctrl_values = []
for item in ctrl_keys:
    plate, well = item.split('_')  # Splitting the item into two parts
    # Finding rows where 9th and 10th columns match the parts
    match = df_adt[(df_adt.iloc[:, 9] == well) & (df_adt.iloc[:, 10] == plate)]
    # Adding the indices of matching rows to the list
    match_index = match.index.tolist()[0]
    # get the NDB to be the key
    ndb = df_adt.iloc[match_index, 0].split('_i')[0]
    ctrl_values.append(ndb)

In [15]:
ctrl_dict = dict(zip(ctrl_keys, ctrl_values))

In [16]:
ctrl_dict

{'P2_F5': 'NDB_899',
 'P2_F11': 'NDB_452',
 'P2_G5': 'NDB_881',
 'P2_G11': 'NDB_665',
 'P2_H5': 'NDB_935',
 'P3_C5': 'NDB_929',
 'P3_A5': 'NDB_557',
 'P3_A11': 'NDB_779',
 'P3_B5': 'NDB_977',
 'P3_B11': 'NDB_341',
 'P2_F6': 'NDB_816',
 'P2_F12': 'NDB_618',
 'P2_G6': 'NDB_498',
 'P2_G12': 'NDB_396',
 'P2_H6': 'NDB_660',
 'P3_C6': 'NDB_933',
 'P3_A6': 'NDB_459',
 'P3_A12': 'NDB_879',
 'P3_B6': 'NDB_594',
 'P3_B12': 'NDB_372'}

# find the matching regions

Cy5 example:

H54_Cy5 -> P2_F5 -> NDB899

-> test1_chr10_01270 

-> Variant1__W1B03__W1B06__NDB_335__NDB_487__NDB_487__NDB_780__NDB_619__NDB_1060__chr10_01270_25kb_pb_31752881 (sequential)

-> the sequential bit is NDB_487

-> P1_F1

-> H21_A750

The other 4 comb bits:

NDB_335 -> P2C2
NDB_780 -> P2C12
NDB_619 -> P2C10
NDB_1060 -> P2C4

In [49]:
for ndb in dic_code_final['chr10_01270']:
    print(dic_readout_seqs[ndb])

('NDB_487', 'tagatagagacgttcgtgaa', 0.0)
('NDB_487', 'tagatagagacgttcgtgaa', 0.0)
('NDB_335', 'aaaccggtcgatacgtgcca', 1.0)
('NDB_1060', 'tggacagctcgcaattctcg', 1.0)
('NDB_619', 'cacatgttgaacgcggccga', 1.0)
('NDB_780', 'ccgtagctgccattacgttt', 1.0)


Cy3 example:

H56_Cy3 -> P2_F12 -> NDB_618

-> chr14_03620

-> Variant1__W1B03__W1B06__NDB_461__NDB_844__NDB_1035__NDB_844__NDB_536__NDB_771__chr14_03620_25kb_pb_90502001

-> seq: NDB_844 -> P1 D4 -> H14_a750

-> comb: 
NDB_1035
NDB_536
NDB_461
NDB_771

In [51]:
for ndb in dic_code_final['chr14_03620']:
    print(dic_readout_seqs[ndb])

('NDB_844', 'attgatagacgatccagacg', 0.0)
('NDB_844', 'attgatagacgatccagacg', 0.0)
('NDB_1035', 'acgggtgtcagacacttact', 1.0)
('NDB_536', 'acggtgcacctcactcgata', 1.0)
('NDB_461', 'ctacatgttaggcgttggtc', 1.0)
('NDB_771', 'cgatacgatcgctaacctac', 1.0)


In [19]:
lib_names,lib_seqs = lt.fastaread(r'/mnt/tscc/wed009/WholeGenome/data/design/LibWGLpilot.fasta')
## write a function to automate this
def get_matching_regions(ctrl_plate_well, 
                         ctrl_dict = ctrl_dict, lib_names = lib_names,
                        dic_code_final = dic_code_final,
                        dic_readout_seqs = dic_readout_seqs):
    # P2_F12 -> NDB_618
    ctrl_ndb = ctrl_dict[ctrl_plate_well]    
    # -> chr14_03620
    ## get the first matching test1 probe
    ctrl_probe = [line for line in lib_names if ctrl_ndb in line][0]
    region = '_'.join(ctrl_probe.split('__')[4].split('_')[:2])
    # -> Variant1__W1B03__W1B06__NDB_461__NDB_844__NDB_1035__NDB_844__NDB_536__NDB_771__chr14_03620_25kb_pb_90502001
    bits = dic_code_final[region]
    adts = [dic_readout_seqs[ndb] for ndb in bits]
    return adts, region

In [20]:
bits_tuple, bits_region = get_matching_regions('P2_F12')
bits_tuple, bits_region

([('NDB_844', 'attgatagacgatccagacg', 0.0),
  ('NDB_844', 'attgatagacgatccagacg', 0.0),
  ('NDB_1035', 'acgggtgtcagacacttact', 1.0),
  ('NDB_536', 'acggtgcacctcactcgata', 1.0),
  ('NDB_461', 'ctacatgttaggcgttggtc', 1.0),
  ('NDB_771', 'cgatacgatcgctaacctac', 1.0)],
 'chr14_03620')

In [21]:
# NDB_784 -> P1_A1
ndb_keys = [x.split('_i')[0] for x in df_adt[0]]
ndb_values = (df_adt[10] + '_' + df_adt[9]).values
ndb_dict = dict(zip(ndb_keys, ndb_values))

In [22]:
ndb_dict

{'NDB_784': 'P1_A1',
 'NDB_755': 'P1_A2',
 'NDB_759': 'P1_A3',
 'NDB_826': 'P1_A4',
 'NDB_578': 'P1_A5',
 'NDB_795': 'P1_A6',
 'NDB_376': 'P1_A7',
 'NDB_713': 'P1_A8',
 'NDB_777': 'P1_A9',
 'NDB_865': 'P1_A10',
 'NDB_533': 'P1_A11',
 'NDB_801': 'P1_A12',
 'NDB_817': 'P1_B1',
 'NDB_725': 'P1_B2',
 'NDB_807': 'P1_B3',
 'NDB_652': 'P1_B4',
 'NDB_710': 'P1_B5',
 'NDB_690': 'P1_B6',
 'NDB_718': 'P1_B7',
 'NDB_971': 'P1_B8',
 'NDB_699': 'P1_B9',
 'NDB_847': 'P1_B10',
 'NDB_743': 'P1_B11',
 'NDB_936': 'P1_B12',
 'NDB_643': 'P1_C1',
 'NDB_683': 'P1_C2',
 'NDB_456': 'P1_C3',
 'NDB_760': 'P1_C4',
 'NDB_815': 'P1_C5',
 'NDB_738': 'P1_C6',
 'NDB_400': 'P1_C7',
 'NDB_965': 'P1_C8',
 'NDB_750': 'P1_C9',
 'NDB_790': 'P1_C10',
 'NDB_464': 'P1_C11',
 'NDB_825': 'P1_C12',
 'NDB_778': 'P1_D1',
 'NDB_431': 'P1_D2',
 'NDB_768': 'P1_D3',
 'NDB_844': 'P1_D4',
 'NDB_554': 'P1_D5',
 'NDB_852': 'P1_D6',
 'NDB_961': 'P1_D7',
 'NDB_917': 'P1_D8',
 'NDB_762': 'P1_D9',
 'NDB_661': 'P1_D10',
 'NDB_728': 'P1_D11',
 '

In [32]:
def find_seq_from_ndb(bits, ndb_dict = ndb_dict):
    '''translate the NDB back to plate number '''
    # find the sequential bit
    seq_bit = [bit for bit in bits if bit[2] == 0][0]
    seq_ndb = seq_bit[0]
    seq_plate_well = ndb_dict[seq_ndb]
    return seq_ndb, seq_plate_well

In [33]:
find_seq_from_ndb(bits_tuple)

('NDB_432', 'P1_F6')

In [34]:
## run through entire control list
seq_plate_well_all = []
seq_ndbs = []
bit_regions = []
for ctrl_plate_well in ctrl_keys:
    bits_tuple, bit_region = get_matching_regions(ctrl_plate_well)
    seq_ndb, seq_plate_well = find_seq_from_ndb(bits_tuple)
    seq_ndbs.append(seq_ndb)
    seq_plate_well_all.append(seq_plate_well)
    bit_regions.append(bit_region)
    print(f'ctrl: {ctrl_plate_well}, seq: {seq_plate_well}')

ctrl: P2_F5, seq: P1_F1
ctrl: P2_F11, seq: P1_B9
ctrl: P2_G5, seq: P1_C7
ctrl: P2_G11, seq: P1_D5
ctrl: P2_H5, seq: P1_F11
ctrl: P3_C5, seq: P1_F11
ctrl: P3_A5, seq: P1_F1
ctrl: P3_A11, seq: P1_B9
ctrl: P3_B5, seq: P1_D5
ctrl: P3_B11, seq: P1_F1
ctrl: P2_F6, seq: P1_C6
ctrl: P2_F12, seq: P1_D4
ctrl: P2_G6, seq: P1_G8
ctrl: P2_G12, seq: P1_C2
ctrl: P2_H6, seq: P1_G4
ctrl: P3_C6, seq: P1_G4
ctrl: P3_A6, seq: P1_G4
ctrl: P3_A12, seq: P1_D10
ctrl: P3_B6, seq: P1_F6
ctrl: P3_B12, seq: P1_F6


In [121]:
pd.Series(seq_plate_well_all).value_counts()

P1_F1     3
P1_G4     3
P1_B9     2
P1_D5     2
P1_F11    2
P1_F6     2
P1_C7     1
P1_C6     1
P1_D4     1
P1_G8     1
P1_C2     1
P1_D10    1
Name: count, dtype: int64

# Export to a csv

In [36]:
df_ctrl_seq = pd.DataFrame({'ctrl_plate_well': ctrl_keys, 
              'ctrl_ndbs':ctrl_values, 
              'seq_plate_well' : seq_plate_well_all,
              'seq_ndb': seq_ndbs, 
              'region' : bit_regions})
df_ctrl_seq

,ctrl_plate_well,ctrl_ndbs,seq_plate_well,seq_ndb,region
0,P2_F5,NDB_899,P1_F1,NDB_487,chr10_01270
1,P2_F11,NDB_452,P1_B9,NDB_699,chr10_03150
2,P2_G5,NDB_881,P1_C7,NDB_400,chr10_03160
3,P2_G11,NDB_665,P1_D5,NDB_554,chr10_03170
4,P2_H5,NDB_935,P1_F11,NDB_860,chr10_05120
5,P3_C5,NDB_929,P1_F11,NDB_860,chr11_04530
6,P3_A5,NDB_557,P1_F1,NDB_487,chr11_00680
7,P3_A11,NDB_779,P1_B9,NDB_699,chr11_02560
8,P3_B5,NDB_977,P1_D5,NDB_554,chr11_02580
9,P3_B11,NDB_341,P1_F1,NDB_487,chr11_04520


In [37]:
save_folder = '/mnt/tscc/wed009/WholeGenome/data/FromBlair/V1_pilot_seq'

In [39]:
df_ctrl_seq.to_csv(os.path.join(save_folder, 'df_ctrl_seq_region_v1_pilot.csv'))

# DEBUG below

In [34]:
df_pilot

,Unnamed: 0.1,Unnamed: 0,block,bin,subblock,batch,locus,Bit_000,Bit_001,Bit_002,...,Bit_1270,Bit_1271,Bit_1272,Bit_1273,Bit_1274,Bit_1275,Bit_1276,Bit_1277,Bit_1278,Bit_1279
0,0,0,0,0,1,0,chr1_00000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,0,1,1,0,chr1_00001,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,2,0,2,1,0,chr1_00002,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,3,0,3,1,0,chr1_00003,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,4,0,4,1,0,chr1_00004,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7795,123895,123895,2064,55,4,0,chrY_01714,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7796,123896,123896,2064,56,4,0,chrY_01715,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7797,123897,123897,2064,57,4,0,chrY_01716,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7798,123898,123898,2064,58,4,0,chrY_01717,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
df_pilot_codebook.shape

(7800, 1280)

In [37]:
df_pilot_codebook

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
column_to_row_indices = {}  # Dictionary to store results
for col_index in ctrl_values:
    # Find row indices where the column value is 1
    row_indices = np.where(df_pilot_codebook[:, col_index] == 1)[0]
    column_to_row_indices[col_index] = row_indices.tolist()

In [41]:
df_pilot.iloc[[2,3]]

,Unnamed: 0.1,Unnamed: 0,block,bin,subblock,batch,locus,Bit_000,Bit_001,Bit_002,...,Bit_1270,Bit_1271,Bit_1272,Bit_1273,Bit_1274,Bit_1275,Bit_1276,Bit_1277,Bit_1278,Bit_1279
2,2,2,0,2,1,0,chr1_00002,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,3,0,3,1,0,chr1_00003,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
df_pilot.iloc[np.where(df_pilot_codebook[:, 160] == 1)]

,Unnamed: 0.1,Unnamed: 0,block,bin,subblock,batch,locus,Bit_000,Bit_001,Bit_002,...,Bit_1270,Bit_1271,Bit_1272,Bit_1273,Bit_1274,Bit_1275,Bit_1276,Bit_1277,Bit_1278,Bit_1279
0,0,0,0,0,1,0,chr1_00000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60,960,960,16,0,1,0,chr1_00960,0,0,0,...,0,0,0,0,0,0,0,0,0,0
120,1920,1920,32,0,1,0,chr1_01920,0,0,0,...,0,0,0,0,0,0,0,0,0,0
180,2880,2880,48,0,1,0,chr1_02880,0,0,0,...,0,0,0,0,0,0,0,0,0,0
240,3840,3840,64,0,1,0,chr1_03840,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7500,120000,120000,2000,0,1,0,chrX_03989,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7560,120960,120960,2016,0,1,0,chrX_04949,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7620,121920,121920,2032,0,1,0,chrX_05909,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7680,122880,122880,2048,0,1,0,chrY_00699,0,0,0,...,0,0,0,0,0,0,0,0,0,0
